# Snowflake Intelligence Hands-On Lab

In this session, you will learn how to:
- Perform semantic modeling to enable text-to-sql conversion via Cortex Analyst
- Build custom tools like web search
- Create Cortex Agents that can use Cortex Analyst and custom tools
- Use Snowflake Intelligence as a UI layer for Cortex Agents
- Investigate Snowflake's built in observability for the agent

## Section 1: Data prep

In this section, we'll configure the necessary Snowflake objects and data to perform the rest of the hands on lab.

For our structured data, we'll use a simple star schema, mocking insurance data. Our model will be composed of the following tables:
- fact_claims: our event table tracking claims that have taken place
- dim_policies: contains our information about the various policies issued
- dim_businesses: contains information about the various businesses that are insured

### 1.1 - Create tables

In [ ]:
USE CORTEX_LAB_DB.DATA;

CREATE OR REPLACE TABLE DIM_BUSINESS (
    BUSINESS_KEY INT PRIMARY KEY,
    BUSINESS_NAME VARCHAR(255),
    BUSINESS_CITY VARCHAR(100),
    BUSINESS_STATE VARCHAR(2),
    NAICS_CODE VARCHAR(10),
    INDUSTRY_SEGMENT VARCHAR(100),
    ASSET_VALUE_MILLIONS NUMERIC(10, 2)
);

CREATE OR REPLACE TABLE DIM_POLICY (
    POLICY_KEY INT PRIMARY KEY,
    BUSINESS_KEY INT REFERENCES DIM_BUSINESS (BUSINESS_KEY),
    POLICY_ID VARCHAR(50),
    POLICY_TYPE VARCHAR(50),
    EFFECTIVE_DATE DATE,
    EXPIRATION_DATE DATE,
    POLICY_STATUS VARCHAR(20), 
    ANNUAL_PREMIUM_USD NUMERIC(12, 2),
    PREMIUM_EARNED_USD NUMERIC(12, 2)
);

CREATE OR REPLACE TABLE FACT_CLAIMS (
    POLICY_KEY INT REFERENCES DIM_POLICY (POLICY_KEY),
    BUSINESS_KEY INT REFERENCES DIM_BUSINESS (BUSINESS_KEY),
    CLAIM_ID VARCHAR(50),
    CLAIM_DATE DATE,
    LOSS_TYPE VARCHAR(100),
    REPORTED_LAG_DAYS INT,
    INCURRED_AMOUNT_USD NUMERIC(12, 2),
    IS_CATASTROPHE BOOLEAN,
    CLAIM_STATUS VARCHAR(20)
);

### 1.2 - Insert data

In [ ]:
INSERT INTO DIM_BUSINESS (BUSINESS_KEY, BUSINESS_NAME, BUSINESS_CITY, BUSINESS_STATE, NAICS_CODE, INDUSTRY_SEGMENT, ASSET_VALUE_MILLIONS)
VALUES
(1, 'Deeper Roots Coffee', 'Cincinnati', 'OH', '722515', 'Restaurant & Hospitality', 2.50),
(2, 'The Eagle OTR', 'Cincinnati', 'OH', '722511', 'Restaurant & Hospitality', 4.10),
(3, 'Rhinegeist Brewery', 'Cincinnati', 'OH', '312120', 'Beverage Manufacturing', 45.00),
(4, 'Cincinnati Zoo & Botanical Garden', 'Cincinnati', 'OH', '712130', 'Arts, Entertainment', 120.00),
(5, 'Local Food Truck Corp.', 'Cincinnati', 'OH', '722330', 'Mobile Food Services', 0.85),
(6, 'Queen City T-Shirt Printing', 'Cincinnati', 'OH', '323113', 'Small-Scale Manufacturing', 3.20),
(7, 'The Lytle Park Hotel', 'Cincinnati', 'OH', '721110', 'Hospitality', 65.00),
(8, 'Cincinnati Museum Center', 'Cincinnati', 'OH', '712110', 'Museums & Historical Sites', 95.50);


INSERT INTO DIM_POLICY (POLICY_KEY, BUSINESS_KEY, POLICY_ID, POLICY_TYPE, EFFECTIVE_DATE, EXPIRATION_DATE, POLICY_STATUS, ANNUAL_PREMIUM_USD, PREMIUM_EARNED_USD)
VALUES
(101, 1, 'GL00101', 'General Liability', '2024-11-20', '2025-11-20', 'Active', 12000.00, 12000.00), 
(102, 2, 'GL00102', 'General Liability', '2024-11-20', '2025-11-20', 'Active', 18000.00, 18000.00), 
(103, 3, 'PC00103', 'Commercial Property', '2025-01-01', '2026-01-01', 'Active', 55000.00, 45833.33),
(104, 4, 'WC00104', 'Workers Compensation', '2025-06-01', '2026-06-01', 'Active', 75000.00, 25000.00),
(105, 5, 'GL00105', 'General Liability', '2024-11-01', '2025-11-01', 'Active', 4500.00, 4500.00),
(106, 6, 'GL00106', 'General Liability', '2025-02-01', '2026-02-01', 'Active', 8500.00, 6375.00),
(107, 7, 'PC00107', 'Commercial Property', '2025-04-01', '2026-04-01', 'Active', 95000.00, 55416.67),
(108, 8, 'CY00108', 'Cyber Liability', '2025-01-01', '2026-01-01', 'Active', 15000.00, 12500.00);

INSERT INTO FACT_CLAIMS (POLICY_KEY, BUSINESS_KEY, CLAIM_ID, CLAIM_DATE, LOSS_TYPE, REPORTED_LAG_DAYS, INCURRED_AMOUNT_USD, IS_CATASTROPHE, CLAIM_STATUS)
VALUES
(101, 1, 'C10001', '2025-03-10', 'Slip & Fall', 1, 8000.00, FALSE, 'Closed'),
(101, 1, 'C10002', '2025-05-05', 'Property Damage (burst pipe)', 3, 5500.00, FALSE, 'Closed'),
(101, 1, 'C10003', '2025-08-25', 'Customer Injury (Hot beverage)', 1, 4500.00, FALSE, 'Open'),
(102, 2, 'C20001', '2025-01-05', 'Food Poisoning (Single incident)', 2, 10000.00, FALSE, 'Closed'),
(102, 2, 'C20002', '2025-04-10', 'Liquor Liability', 10, 15000.00, FALSE, 'Open'),
(102, 2, 'C20003', '2025-07-20', 'Theft (patio furniture)', 4, 2000.00, FALSE, 'Closed'),
(103, 3, 'C30001', '2025-03-20', 'Minor Fire Damage', 7, 12000.00, FALSE, 'Closed'),
(104, 4, 'C40001', '2025-07-01', 'Workplace Minor Injury', 2, 5500.00, FALSE, 'Closed'),
(105, 5, 'C50001', '2025-02-05', 'Vehicle Collision', 5, 2500.00, FALSE, 'Open'),
(106, 6, 'C60001', '2025-05-01', 'Equipment Failure', 3, 8000.00, FALSE, 'Open'),
(107, 7, 'C70001', '2025-08-03', 'Small Hail Damage', 2, 15000.00, FALSE, 'Open'),
(108, 8, 'C80001', '2025-09-10', 'System Ransomware', 1, 10000.00, FALSE, 'Open');

### 1.3 - Run some sample queries

In [ ]:
-- Amount of loss incurred from claims for Commerical Property policies
SELECT policy_type, sum(incurred_amount_usd)
FROM fact_claims c
JOIN dim_policy p
  on c.policy_key = p.policy_key
WHERE policy_type = 'Commercial Property'
GROUP BY policy_type
;


## Section 2: Cortex Analyst

Cortex Analyst is a fully managed, AI-powered feature in Snowflake to answer natural language queries about your structured data. It relies on a Semantic model to provide metadata and context about your relational data to the LLM so that it can generate SQL based on the question at hand.

The Semantic Layer can be defined as a YAML file or as a Semantic View - a new schema-level object in Snowflake.


### 2.1 - Semantic Views
Semantic Views are schema-level objects in Snowflake that enable us to define business metrics, entities, and their relationships. This context will be leveraged by Cortex Analyst for generating SQL to answer natural language questions.

Semantic Views are comprised of the following components:

- **Tables**: logical tables that map to Snowflake tables or views
- **Facts**: row-level attributes tied to a logical table that represent specific business events or transactions
- **Dimensions**: categorical attributes tied to a logical table that gives meaning to metrics by grouping data into meaningful categories
- **Metrics**: quantifiable measures of business performance calculated by aggregating facts or other columns from the same table
**Named filters**: logic to filter a logical table based on some business rule
**Relationships**: how logical tables are mapped to one another. This enables Cortex Analyst to join multiple logical tables together

Semantic Views can be created through SQL (as seen below), but also through the Snowsight UI which is what we'll walk through together in this lab.

In [ ]:
-- Semantic View SQL - We will create this together via the UI
create or replace semantic view CORTEX_LAB_DB.DATA.RISK_SEMANTIC_VIEW
	tables (
		DIM_BUSINESS primary key (BUSINESS_KEY) comment='This table stores information about businesses, including their unique identifier, name, location, industry classification, and financial data.',
		DIM_POLICY primary key (POLICY_KEY) comment='This table stores information about insurance policies, including policy details, coverage periods, and financial data. It serves as a dimension table in a data warehouse, providing a single source of truth for policy data and enabling analysis and reporting across various business dimensions.',
		FACT_CLAIMS comment='This table stores information about insurance claims, including the policy and business associated with the claim, the date the claim was made, the type of loss, the number of days between the incident and the claim report, the amount of the claim in US dollars, whether the claim is related to a catastrophe, and the current status of the claim.'
	)
	relationships (
		CLAIMS_TO_BUSINESS as FACT_CLAIMS(BUSINESS_KEY) references DIM_BUSINESS(BUSINESS_KEY),
		CLAIMS_TO_POLICY as FACT_CLAIMS(POLICY_KEY) references DIM_POLICY(POLICY_KEY)
	)
	facts (
		DIM_BUSINESS.ASSET_VALUE_MILLIONS as ASSET_VALUE_MILLIONS comment='The total value of assets held by the business, expressed in millions of dollars.',
		DIM_BUSINESS.BUSINESS_KEY as BUSINESS_KEY comment='Unique identifier for a business entity, used to link to other tables and provide a distinct reference for each business.',
		DIM_POLICY.ANNUAL_PREMIUM_USD as ANNUAL_PREMIUM_USD comment='The annual premium amount paid by the policyholder in US dollars.',
		DIM_POLICY.BUSINESS_KEY as BUSINESS_KEY comment='Unique identifier for a policy, used to distinguish one policy from another.',
		DIM_POLICY.POLICY_KEY as POLICY_KEY comment='Unique identifier for a policy in the insurance portfolio.',
		DIM_POLICY.PREMIUM_EARNED_USD as PREMIUM_EARNED_USD comment='The total amount of premium earned by the policy in US dollars.',
		FACT_CLAIMS.BUSINESS_KEY as BUSINESS_KEY comment='Unique identifier for a business entity, used to link claims data to the corresponding business information.',
		FACT_CLAIMS.INCURRED_AMOUNT_USD as INCURRED_AMOUNT_USD comment='The total amount of financial loss incurred by the insurance company for a claim, expressed in US dollars.',
		FACT_CLAIMS.POLICY_KEY as POLICY_KEY comment='Unique identifier for a policy, used to link claims to the corresponding policy details.',
		FACT_CLAIMS.REPORTED_LAG_DAYS as REPORTED_LAG_DAYS comment='The number of days between the date an incident occurred and the date it was reported.'
	)
	dimensions (
		DIM_BUSINESS.BUSINESS_CITY as BUSINESS_CITY comment='The city where the business is located.',
		DIM_BUSINESS.BUSINESS_NAME as BUSINESS_NAME comment='The name of the business or organization.',
		DIM_BUSINESS.BUSINESS_STATE as BUSINESS_STATE comment='The state in which the business is located.',
		DIM_BUSINESS.INDUSTRY_SEGMENT as INDUSTRY_SEGMENT comment='The industry segment in which the business operates, such as manufacturing, services, or entertainment.',
		DIM_BUSINESS.NAICS_CODE as NAICS_CODE comment='A 6 digit code used to classify businesses based on their economic activity.',
		DIM_POLICY.EFFECTIVE_DATE as EFFECTIVE_DATE comment='The date when a policy becomes effective and its terms and conditions are applied.',
		DIM_POLICY.EXPIRATION_DATE as EXPIRATION_DATE comment='Date on which a policy is set to expire.',
		DIM_POLICY.POLICY_ID as POLICY_ID comment='Unique identifier for a policy in the insurance portfolio.',
		DIM_POLICY.POLICY_STATUS as POLICY_STATUS comment='Indicates the current status of a policy, with "Active" signifying that the policy is currently in effect and being enforced.',
		DIM_POLICY.POLICY_TYPE as POLICY_TYPE comment='Type of insurance policy held by the customer, such as workers compensation, cyber liability, or general liability.',
		FACT_CLAIMS.CLAIM_DATE as CLAIM_DATE comment='Date on which the claim was made.',
		FACT_CLAIMS.CLAIM_ID as CLAIM_ID comment='Unique identifier for a claim submitted by a customer.',
		FACT_CLAIMS.CLAIM_STATUS as CLAIM_STATUS comment='The current status of a claim, indicating whether it is still being processed (Open) or has been resolved and completed (Closed).',
		FACT_CLAIMS.IS_CATASTROPHE as IS_CATASTROPHE comment='Indicates whether the claim is related to a catastrophic event, such as a natural disaster, or not.',
		FACT_CLAIMS.LOSS_TYPE as LOSS_TYPE comment='The type of loss or damage that occurred, such as an accident, injury, or property damage, which resulted in a claim being filed.'
	)
	with extension (CA='{"tables":[{"name":"DIM_BUSINESS","dimensions":[{"name":"BUSINESS_CITY","sample_values":["Cincinnati"]},{"name":"BUSINESS_NAME","sample_values":["Deeper Roots Coffee","The Eagle OTR","Rhinegeist Brewery"]},{"name":"BUSINESS_STATE","sample_values":["OH"]},{"name":"INDUSTRY_SEGMENT","sample_values":["Beverage Manufacturing","Restaurant & Hospitality","Arts, Entertainment"]},{"name":"NAICS_CODE","sample_values":["722511","312120","722515"]}],"facts":[{"name":"ASSET_VALUE_MILLIONS","sample_values":["4.10","2.50","45.00"]},{"name":"BUSINESS_KEY","sample_values":["1","2","3"]}],"filters":[{"name":"IS_ACCOMODATION_AND_FOOD_SERVICES","synonyms":["accommodation_and_catering","accommodation_services","food_and_beverage_services","food_services","hospitality_industry","hotel_and_restaurant_services","lodging_and_food"],"description":"Indicates whether the business operates in the Accommodation and Food Services sector, which includes establishments providing lodging, meals, and beverages for immediate consumption.","expr":"NAICS_CODE like ''72%''"}]},{"name":"DIM_POLICY","dimensions":[{"name":"POLICY_ID","sample_values":["PC00103","GL00101","GL00102"]},{"name":"POLICY_STATUS","sample_values":["Active"]},{"name":"POLICY_TYPE","sample_values":["Workers Compensation","Cyber Liability","General Liability"]}],"facts":[{"name":"ANNUAL_PREMIUM_USD","sample_values":["18000.00","12000.00","8500.00"]},{"name":"BUSINESS_KEY","sample_values":["1","2","3"]},{"name":"POLICY_KEY","sample_values":["101","102","103"]},{"name":"PREMIUM_EARNED_USD","sample_values":["18000.00","12000.00","45833.33"]}],"filters":[{"name":"ACTIVE_POLICIES","synonyms":["active_policy_count","current_policies","existing_policies","in_force_policies","live_policies","policies_in_force"],"description":"Indicates whether a policy is currently active or not.","expr":"EXPIRATION_DATE >= CURRENT_DATE"},{"name":"DUE_FOR_RENEWAL_SOON","synonyms":["approaching_renewal","close_to_renewal","near_renewal","renewal_due_soon","renewal_pending","soon_to_expire","upcoming_renewal"],"description":"Active policies that are expiring in the next 30 days.","expr":"EXPIRATION_DATE BETWEEN CURRENT_DATE AND DATEADD(DAY, 30, CURRENT_DATE)"}],"time_dimensions":[{"name":"EFFECTIVE_DATE","sample_values":["2024-11-20","2024-11-01","2025-01-01"]},{"name":"EXPIRATION_DATE","sample_values":["2026-01-01","2026-06-01","2025-11-20"]}]},{"name":"FACT_CLAIMS","dimensions":[{"name":"CLAIM_ID","sample_values":["C10003","C10001","C10002"]},{"name":"CLAIM_STATUS","sample_values":["Open","Closed"]},{"name":"IS_CATASTROPHE","sample_values":["FALSE"]},{"name":"LOSS_TYPE","sample_values":["Slip & Fall","Minor Fire Damage","Customer Injury (Hot beverage)"]}],"facts":[{"name":"BUSINESS_KEY","sample_values":["1","2","3"]},{"name":"INCURRED_AMOUNT_USD","sample_values":["4500.00","5500.00","8000.00"]},{"name":"POLICY_KEY","sample_values":["101","102","103"]},{"name":"REPORTED_LAG_DAYS","sample_values":["1","2","3"]}],"time_dimensions":[{"name":"CLAIM_DATE","sample_values":["2025-05-05","2025-08-03","2025-03-10"]}]}],"relationships":[{"name":"CLAIMS_TO_BUSINESS"},{"name":"CLAIMS_TO_POLICY"}]}');

In [ ]:
-- Verify Semantic View
SHOW SEMANTIC VIEWS;

### 2.2 - Creating our first Cortex Agent
Now that we have a Semantic View created for use with Cortex Analyst, we can create a Cortex Agent that will be able to answer natural language questions about our data.

Cortex Agents are the brains of this demo. Agents will orchestrate both structured and unstructured data to deliver insights. They plan tasks, use tools to execute these tasks, and generate responses. Agents can be configured to use Cortex Analyst (structured data), Cortex Search (unstructured data), and custom tools to generate answers.



In [ ]:
CREATE OR REPLACE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.Risk_Analysis_Agent_Structured
WITH PROFILE='{"display_name": "1. Risk Analysis Agent - Structured Data"}'
COMMENT=$$ This is an agent that can answer questions about our insurance data including policies, businesses, and claims. $$
FROM SPECIFICATION $$
{
  "instructions": {
    "response": "You are a data analyst who has access to insurance policies, insured businesses, and claims. If user does not specify a date range assume it for year 2025. Leverage data from all domains to analyze & answer user questions. Provide visualizations if possible. Trendlines should default to linecharts, Categories default to Barchart.",
    "orchestration": "Use cortex analyst for any questions regarding the policies, businesses, and claims data.",
    "sample_questions": [
      {
        "question": "What has been our incurred loss by category over the last 12 months?",
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Insurance Datamart",
        "description": "Allows users to query finance data for a company in terms of revenue & expenses."
      }
    }
  ],
  "tool_resources": {
    "Query Insurance Datamart": {
      "semantic_view": "CORTEX_LAB_DB.DATA.RISK_SEMANTIC_VIEW",
      "execution_environment": {
          "query_timeout": 0,
          "type": "warehouse",
          "warehouse": "SNOW_INTELLIGENCE_DEMO_WH"
      },
    }
  }
}
$$;


### 2.3 Test Analyst Agent v1!

Open Snowflake Intelligence by clicking on 'AI & ML' on the left navbar and then choosing 'Snowflake Intelligence'.

Once there, ask away about our structured data! Here are some sample questions as thought starters:

- What was our incurred loss amount by policy type over the last 12 months?
- How many claims have been made over the last 3 months? 


## Section 3: Custom Tools

In addition to Cortex Analyst and Cortex Search, we can also equip our agent with custom tools for execution. In this section, we'll create additional tools for sending emails and performing web search.

### 3.2 Web Search Tool
We can also create functions using python and make those available as tools to the agent. The code below creates a simple web search function that can be used by our agent for getting additional information from the web.

In [ ]:
CREATE OR REPLACE FUNCTION Web_search(search_query STRING)
RETURNS VARIANT
LANGUAGE PYTHON
RUNTIME_VERSION = 3.11
ARTIFACT_REPOSITORY = snowflake.snowpark.pypi_shared_repository
HANDLER = 'web_search'
EXTERNAL_ACCESS_INTEGRATIONS = (Snowflake_intelligence_ExternalAccess_Integration)
PACKAGES = ('tavily-python')
AS
$$
from tavily import TavilyClient

def web_search(search_query: str):
    try:
        TAVILY_API_KEY = "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"
        tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

        response = tavily_client.search(
            query=search_query,
            max_results=5,
            include_answer=True
        )

        results = response.get("results", [])
        if not results:
            return {"summary": "No results found.", "results": [], "markdown_sources": ""}

        formatted_results = []
        markdown_source_list = []
        
        for i, result in enumerate(results[:3]):
            title = result.get("title", "Untitled")
            url = result.get("url", "")
            snippet = result.get("content", "").strip()[:1500]

            formatted_results.append({
                "rank": i + 1,
                "title": title,
                "url": url,
                "snippet": snippet
            })
            
            if url and title:
                markdown_source_list.append(f"* [{title}]({url})") 
        
        markdown_sources_string = "\n".join(markdown_source_list)

        return {
            "summary": response.get("answer", ""),
            "results": formatted_results,
            "markdown_sources": markdown_sources_string
        }

    except Exception as e:
        return {"error": str(e)}
$$;

### 3.3 Update the Agent with tools
Now that we have the tools defined, we can recreate our agent to give it the ability to call the tools.

In [ ]:
# TODO: Update prompts to include instructions on using our web search tool
agent_response_instructions = """
You are a data analyst who has access to insurance policies, insured businesses, and claims. 
If user does not specify a date range assume it for year 2025. 
Leverage data from all domains to analyze & answer user questions. 
Provide visualizations if possible. Trendlines should default to linecharts, Categories default to Barchart.

<add web search instructions here>
"""
agent_response_orchestration = """
Use cortex analyst for any questions regarding the policies, businesses, and claims data.

<add web search instructions here>
"""

In [ ]:
import json
from snowflake.snowpark.context import get_active_session
session = get_active_session()

create_agent_sql = f"""
CREATE OR REPLACE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.Risk_Analysis_Agent_Tools
WITH PROFILE='{{"display_name": "2. Risk Analysis Agent - Tools"}}'
COMMENT=$$ This is an agent that can answer questions about our insurance data including policies, businesses, and claims. has access to web search via tools. $$
FROM SPECIFICATION $$
{{  
  "instructions": {{
    "response": {json.dumps(agent_response_instructions)},
    "orchestration": {json.dumps(agent_response_orchestration)},
    "sample_questions": [
      {{
        "question": "Show me all active policies for our clients in Cincinnati's Restaurant & Hospitality sector with a Loss Ratio above 1.4. Also provide their count of claims this year and the business name"
      }}
    ]
  }},
  "tools": [
    {{
      "tool_spec": {{
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Insurance Datamart",
        "description": "Allows users to query finance data for a company in terms of revenue & expenses."
      }}
    }},
    {{
      "tool_spec": {{
        "type": "generic",
        "name": "Web_search",
        "description": "Search the web for recent or relevant information about a topic.",
        "input_schema": {{
          "type": "object",
          "properties": {{
            "search_query": {{
              "description": "The user-provided search query.",
              "type": "string"
            }}
          }},
          "required": ["search_query"]
        }}
      }}
    }},
  ],
  "tool_resources": {{
    "Query Insurance Datamart": {{
      "semantic_view": "CORTEX_LAB_DB.DATA.RISK_SEMANTIC_VIEW",
      "execution_environment": {{
          "query_timeout": 0,
          "type": "warehouse",
          "warehouse": "SNOW_INTELLIGENCE_DEMO_WH"
      }},
    }},
    "Web_search": {{
      "execution_environment": {{
        "query_timeout": 0,
        "type": "warehouse",
        "warehouse": "SNOW_INTELLIGENCE_DEMO_WH"
      }},
      "identifier": "CORTEX_LAB_DB.DATA.WEB_SEARCH",
      "name": "WEB_SEARCH(VARCHAR)",
      "type": "function"
    }}
  }}
}}
$$;
"""

session.sql(create_agent_sql).collect()
print("Agent successfully created!")

In [ ]:
# Here's some prompts we created for this. Check these out to see how yours differ!
agent_response_instructions = """
You are a data analyst who has access to insurance policies, insured businesses, and claims. 
If the user does not specify a date range, assume it is for the year 2025.
Leverage data from all domains to analyze and answer user questions. 
Provide visualizations if possible. Trendlines should default to line charts, categories default to bar charts. 

You have access to a Web_search tool that returns structured results, including **a pre-formatted 'markdown_sources' string for citation.** 
**STRICT FORMAT REQUIREMENT: Your response MUST consist of two parts only: 
  1. A short summary of your findings, primarily using the search tool's 'summary' field. 
  2. A 'Sources' section, starting with a Level 2 markdown heading '## Sources', followed immediately by the exact, unedited content of the 'markdown_sources' field.**
     If no search results are found, clearly state 'No results found.' Do not make up URLs or sources or attempt to format the sources manually.
"""

agent_response_orchestration = """
When asked about analytical questions regarding policies, businesses, and claims use Cortex Analyst to answer.
When the user's request involves any of the following, ALWAYS use the Web_search tool:
- Looking up recent events, news, updates, or regulatory actions
- Searching for company information not known to exist in internal data
- Finding information from external or public sources (internet)

If uncertain, prefer using Web_search first. After retrieving web results, do not call any other tools or reasoning steps. 
Generate a single 'LLM Response Generation' output that follows the required format.
"""

### 3.4 Test the updated agent

One sample flow (ask all questions in one conversation):
- Show me all active policies for our clients in Cincinnati's Restaurant & Hospitality sector with a Loss Ratio above 1.4. Also provide their count of claims this year and the business name

**Turn on Research Mode and select the web search tool on "Sources"**
- Use the web search tool to find any recent news or regulatory actions related to health code violations or labor issues about Deeper Roots Coffee in Cincinnati.
- Combine the internal policy data (loss ratio) with the external web search findings and provide a final renewal recommendation for Deeper Roots Coffee. Justify your score.

## Section 4: AI Observability

After asking a few questions to our agent, we can explore Snowflake's built in AI Observability tooling to understand what tools were invoked to accomplish the given tasks.

Start by clicking on 'AI & ML' on the left hand havbar, and then opening 'Cortex Agents' in a new tab. Once on the Cortex Agents screen, choose the 'Risk_Analysis_Agent_Tools' agent. Here, you can see all of the configuration for our agent, including which tools it has available.

Next, click on 'Monitoring' and then choose one of your conversations. This is where we can see all the detail going on behind the scenes. Find a task that used Cortex Analyst - there we can see the exact SQL that was generated and executed to answer the given question. On the 'Web search' tasks, we can see the given prompt and the web search results that was fed back to the model.